In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('A Track 학습셋.csv')
data

,Log_Number,payload,label_action
0,0,"GET /forum1_professionnel.asp?n=/.\\\""./.\\\""....",System_Cmd_Execution
1,1,POST /owa/auth/logon.aspx?replaceCurrent=1%22%...,System_Cmd_Execution
2,2,GET /goods/goods_search?display_type=list&arr_...,SQL_Injection
3,3,GET / HTTP/1.1\r\n\r\n,HOST_Scan
4,4,GET /sub_04_1_read.php?page=1&id=31%29%3BSELEC...,System_Cmd_Execution
...,...,...,...
44995,44995,GET /app/javascript/js/skin-responsive.js?v=%3...,System_Cmd_Execution
44996,44996,GET /board/?category=type+%25SYSTEMROOT%25%5Cw...,System_Cmd_Execution
44997,44997,POST /sub_02_4.php HTTP/1.1\r\nContent-Length:...,SQL_Injection
44998,44998,GET /admin/myadmin HTTP/1.1\r\nHost: lab.colle...,Path_Disclosure


In [3]:
data = data[['payload', 'label_action']]
data

,payload,label_action
0,"GET /forum1_professionnel.asp?n=/.\\\""./.\\\""....",System_Cmd_Execution
1,POST /owa/auth/logon.aspx?replaceCurrent=1%22%...,System_Cmd_Execution
2,GET /goods/goods_search?display_type=list&arr_...,SQL_Injection
3,GET / HTTP/1.1\r\n\r\n,HOST_Scan
4,GET /sub_04_1_read.php?page=1&id=31%29%3BSELEC...,System_Cmd_Execution
...,...,...
44995,GET /app/javascript/js/skin-responsive.js?v=%3...,System_Cmd_Execution
44996,GET /board/?category=type+%25SYSTEMROOT%25%5Cw...,System_Cmd_Execution
44997,POST /sub_02_4.php HTTP/1.1\r\nContent-Length:...,SQL_Injection
44998,GET /admin/myadmin HTTP/1.1\r\nHost: lab.colle...,Path_Disclosure


In [4]:
data = data[(data['label_action'] == 'Path_Disclosure')]
data

,payload,label_action
11,GET /../../../../../../../../../../../../etc/s...,Path_Disclosure
22,GET /company/about/_mmDBScripts/MMHTTPDB.aspc7...,Path_Disclosure
26,GET /%2f..%2f..%2f..%2f..%2f..%2f..%2f..%2f..%...,Path_Disclosure
30,GET /board/board_list?code=%2F..%2Fwinnt%2Fwin...,Path_Disclosure
33,GET /goods/goods_search?display_type=list&arr_...,Path_Disclosure
...,...,...
44956,GET /phpMyAdmin/index.php?lang=en HTTP/1.1\r\n...,Path_Disclosure
44985,GET /phpMyAdmin-2.8.0-beta1/ HTTP/1.1\r\nHost:...,Path_Disclosure
44987,POST /board/?category=&gdviewer=..%2F..%2F..%2...,Path_Disclosure
44993,GET /admin/captcha/securimage_show?160c3e6d29e...,Path_Disclosure


In [5]:
data.to_json('Path_Disclosure.json', 'records')

In [6]:
import json

In [7]:
with open('Path_Disclosure.json', 'r') as f:
    data = json.load(f)

In [8]:
import urllib.parse  #URL 처리와 관련된 여러 유용한 함수와 클래스 제공

def unquote(text):
    k = 0               #반복 회수를 세기 위한 변수 k
    uq_prev = text      #uq_prev text 인자 받은거 입력
    while(k < 100):     #반복 작업을 100회 이내로 실행
        uq = urllib.parse.unquote_plus(uq_prev)
        if uq == uq_prev:
            break
        else:
            uq_prev = uq
    
    return uq_prev

def remove_new_line(text):
    text = text.strip()
    text = ' '.join(text.splitlines())
    
    return text

def remove_multiple_whitespaces(text):
    return ' '.join(text.split())

def clean_pattern(pattern):
    pattern = unquote(pattern)                      #url decoding
    pattern = remove_new_line(pattern)              #\n 줄바꿈 문자 제거
    pattern = pattern.lower()                       #소문자로 바꾸는거
    pattern = remove_multiple_whitespaces(pattern)  # 공백 제거
    
    return pattern


def prepare_data(x):
    out = {}            #var dict
    out['pattern'] = clean_pattern(x['payload'])
    
    if x['label_action'] == 'Vulnerability_Scan':
        out['label_action'] = 'Vulnerability_Scan'
    
    return out

In [9]:
Path_Disclosure = list(map(prepare_data, data))
Path_Disclosure

[{'pattern': 'get /../../../../../../../../../../../../etc/shadow http/1.1\\r\\nhost: 10.0.17.21\\r\\nconnection: keep-alive\\r\\nuser-agent: mozilla/5.00 (nikto/2.1.6) (evasions:none) (test:dishwasher)\\r\\n\\r\\n'},
 {'pattern': 'get /company/about/_mmdbscripts/mmhttpdb.aspc75 http/1.1\\r\\nhost: www.college.school\\r\\nauthorization: negotiate tlrmtvntuaabaaaat4ii4gaaaaaaaaaaaaaaaaaaaaakaf1yaaaadw==\\r\\naccept-encoding: gzip, deflate\\r\\nuser-agent: mozilla/5.0 (gecko) arachni/v10.0.0.80\\r\\naccept: text/html,application/xhtml xml,application/xml;q=0.9,*/*;q=0.8\\r\\naccept-language: en-us,en;q=0.8,he;q=0.6\\r\\nx-arachni-scan-seed: 4818b1cc56353e785f9bfe9b055b7183\\r\\ncookie: designart_site=cg0q4hq5j1rgo9o2ns7s3nsnjlauq1rp\\r\\n\\r\\n'},
 {'pattern': 'get //../../../../../../../../../../etc/passwd http/1.1\\r\\ncontent-length: 36\\r\\nhost: 10.0.17.21\\r\\nuser-agent: mozilla/5.00 (nikto/2.1.6) (evasions:none) (test:000490)\\r\\ncontent-type: application/x-www-form-urlencoded\\

In [10]:
with open('Path_Disclosure.json', 'w') as f:
    json.dump(Path_Disclosure, f)